# Eventador example data science workflows

The Eventador Platform helps you manage data for your data science work from simple analytics to machine learning models. In this example we will perform data management tasks and some basic data science tasks to demonstrate various capabilities of the platform.

## Connecting to, defining, and organizing feeds of data in Apache Kafka

Let's first connect to the system, and inspect some Kafka topics/tables and see the datatypes we can work with. This works/looks just like the database you are used to using.

In [7]:
import eventador_python as ev
import pandas as pd


config={'auth': {'username': 'myusername', 'password': 'xxxx'}}
e = ev.EventadorQuery()
e.login(config['auth']['username'], config['auth']['password'])

Getting CSRF token..
Login successful, user is: kgorman


In [2]:
e.command("show tables")

{
    "table_name": "JMO_IN",
    "flavor": "source",
    "type": "kafka",
    "dtcreated": "2019-11-19 21:26:45.082614",
    "id": 738
}

{
    "table_name": "JMO_IN_BREAK",
    "flavor": "source",
    "type": "kafka",
    "dtcreated": "2020-01-09 21:47:21.463287",
    "id": 1193
}

{
    "table_name": "adsbx",
    "flavor": "source",
    "type": "kafka",
    "dtcreated": "2020-01-08 21:21:36.344694",
    "id": 1110
}

{
    "table_name": "adsbx_transform",
    "flavor": "source",
    "type": "kafka",
    "dtcreated": "2020-02-11 22:59:37.670902",
    "id": 2197
}

{
    "table_name": "airplanes",
    "flavor": "source",
    "type": "kafka",
    "dtcreated": "2019-09-06 18:12:10.762355",
    "id": 17
}

{
    "table_name": "airplanes_match_source",
    "flavor": "source",
    "type": "kafka",
    "dtcreated": "2020-03-23 17:32:35.020567",
    "id": 3836
}

{
    "table_name": "airplanes_pager",
    "flavor": "source",
    "type": "kafka",
    "dtcreated": "2020-03-02 23:35:36.209733",

In [3]:
e.command("desc authorizations")

{
    "fields": [
        {
            "doc": "Type inferred from '706'",
            "name": "userid",
            "type": "long"
        },
        {
            "doc": "Type inferred from '21305'",
            "name": "amount",
            "type": "long"
        },
        {
            "doc": "Type inferred from '31.085854'",
            "name": "lat",
            "type": "double"
        },
        {
            "doc": "Type inferred from '-112.757024'",
            "name": "lon",
            "type": "double"
        },
        {
            "doc": "Type inferred from '\"4651725968631115\"'",
            "name": "card",
            "type": "string"
        }
    ],
    "name": "inferredSchema",
    "type": "record"
}



The schema for authorizations is inferred from the topic. We can write some queries against it. The SQL is launched as a *continuous SQL* job that is always running and keeping a view up to date with the latest versions of the data. This view is exposed as a REST endpoint we can use anywhere we need data - from exploration, training, testing, or in production.

This is extremely high performance, we don't wait for results from the data, it's continuously fed to the view and we can access the view as we need to in our models.

If we get the SQL wrong then it gives us instant feedback, we can iterate, we don't need to wait for a runtime failure to happen. You can also use the https://eventador.cloud interface to author and launch jobs.

In [12]:
e.command("set cluster 7.0.0-pre3")
sql = """select userid, lat, lon, amount, card from authorizations where amount > 10000 and card is not null"""
e.query(sql, "kg_job3")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

We can now use the results of that continuously running SQL job in Pandas via the materialized view REST endpoint.

In [ ]:
df = pd.read_json()